# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CePathManager
import pathlib

path_manager = CePathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Ce config file

In [2]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml")
post_proc_cfg = CeCaseConfig.from_file(config_path)

post_proc_cfg

CeCaseConfig(shape_coefficient={'measurement_1': CeConfig(statistics=[BasicStatisticModel(stats='mean', display_name=''), BasicStatisticModel(stats='rms', display_name=''), BasicStatisticModel(stats='skewness', display_name=''), BasicStatisticModel(stats='kurtosis', display_name=''), ParameterizedStatisticModel(stats='mean_eq', display_name='', params=MeanEquivalentParamsModel(scale_factor=0.61)), ParameterizedStatisticModel(stats='min', display_name='', params=ExtremeAbsoluteParamsModel(method_type='Absolute')), ParameterizedStatisticModel(stats='max', display_name='', params=ExtremePeakParamsModel(method_type='Peak', peak_factor=3.0))], zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 5.0, 10.0, 15.0, 20.0]), no_zoning=['t1_ym'], exclude=['p1_xp'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 25.0

Read LNAS Mesh

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Process shape coefficient use case

In [4]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import CeOutput, process_Ce

cp_data_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.normalized.h5")

for cfg_label, cfg in post_proc_cfg.shape_coefficient.items():
    logger.info(f"Processing {cfg_label} ...")

    Ce_output: CeOutput = process_Ce(
        mesh=mesh,
        cfg=cfg,
        cp_path=cp_data_path,
    )

    Ce_output.save_region_info(cfg_label=cfg_label, path_manager=path_manager)
    Ce_output.save_outputs(cfg_label=cfg_label, path_manager=path_manager)
    Ce_output.export_mesh(cfg_label=cfg_label, path_manager=path_manager)

[2024-05-23 15:34:37,977] [INFO] - cfdmod - Processing measurement_1 ... (3797609262.py:7)
[2024-05-23 15:34:37,979] [INFO] - cfdmod - Getting geometry data... (Ce_data.py:151)
[2024-05-23 15:34:37,996] [DEBUG] - cfdmod - Surface p1_xp ignored! (Ce_geom.py:52)
[2024-05-23 15:34:38,034] [INFO] - cfdmod - Tabulating geometry data... (Ce_data.py:154)
[2024-05-23 15:34:38,171] [INFO] - cfdmod - Processing timesteps groups... (Ce_data.py:161)
[2024-05-23 15:34:40,125] [INFO] - cfdmod - Calculating statistics... (Ce_data.py:175)
[2024-05-23 15:34:40,163] [INFO] - cfdmod - Processing surfaces... (Ce_data.py:178)
[2024-05-23 15:34:40,677] [WARNING] - cfdmod - Region refinement is greater than data refinement. Resulted in NaN values (Ce_data.py:104)
[2024-05-23 15:34:40,732] [WARNING] - cfdmod - Region refinement is greater than data refinement. Resulted in NaN values (Ce_data.py:104)
[2024-05-23 15:34:40,838] [WARNING] - cfdmod - Region refinement is greater than data refinement. Resulted in N